In [108]:
# General
import sys
import os
import shutil
import glob
from time import time, strftime
from collections import OrderedDict
od = OrderedDict
from importlib import reload
from glob import glob
import itertools
import h5py

# Scientific
import numpy as np
import pandas as pd
pd.options.display.max_rows = 200
pd.options.display.max_columns = 999
import scipy.io as sio
from scipy.ndimage.filters import gaussian_filter

# Stats
import scipy.stats as stats
import statsmodels.api as sm
import astropy.stats.circstats as circstats
import pycircstat
import random
from sklearn.decomposition import PCA

# Plots
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as patches
mpl.rcParams['grid.linewidth'] = 0.1
mpl.rcParams['grid.alpha'] = 0.75
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['lines.markersize'] = 3
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15 
colors = ['1f77b4', 'd62728', '2ca02c', 'ff7f0e', '9467bd', 
          '8c564b', 'e377c2', '7f7f7f', 'bcbd22', '17becf']
mpl.rcParams['axes.prop_cycle'] = mpl.cycler('color', colors)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.formatter.offset_threshold'] = 2
mpl.rcParams['axes.labelsize'] = 17
mpl.rcParams['axes.labelpad'] = 10
mpl.rcParams['axes.titlesize'] = 19
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.axisbelow'] = True
mpl.rcParams['legend.loc'] = 'upper right'
mpl.rcParams['legend.fontsize'] = 15
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['figure.titlesize'] = 19
mpl.rcParams['figure.figsize'] = (6.85039, 4.79527) 
mpl.rcParams['figure.subplot.wspace'] = 0.25 
mpl.rcParams['figure.subplot.hspace'] = 0.25 
mpl.rcParams['font.sans-serif'] = ['Helvetica']
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['pdf.fonttype'] = 42

import mne
import neurodsp
from ptsa.data.TimeSeriesX import TimeSeries
from ptsa.data.filters import MorletWaveletFilter
import cmlreaders
from cmlreaders import CMLReader

sys.path.append('/home1/dscho/code/general')
import data_io as dio
import array_operations as aop
import spectral_processing as spp
from eeg_plotting import plot_trace, plot_trace2
sys.path.append('/home1/dscho/code/projects/manning_replication')
import manning_utils
import manning_analysis
sys.path.append('/home1/dscho/code/projects/unit_activity_and_hpc_theta')
import phase_locking
import lfp_synchrony
sys.path.insert(0, '/home1/dscho/code/projects/theta_reinstatement')
import pal_reinstatement as palr
sys.path.insert(0, '/home1/esolo/notebooks/codebases/')
import CML_stim_pipeline
from loc_toolbox import get_region, update_pairs

# Colors
n = 4
c = 2
colors = [sns.color_palette('Blues', n)[c], 
          sns.color_palette('Reds', n)[c], 
          sns.color_palette('Greens', n)[c],
          sns.color_palette('Purples', n)[c],
          sns.color_palette('Oranges', n)[c],
          sns.color_palette('Greys', n)[c],
          sns.color_palette('YlOrBr', n+3)[c],
          'k']

In [60]:
vfr_dir = '/home1/dscho/projects/vfr/data/polyn_data'
vfr_f = os.path.join(vfr_dir, 'data_VFR.mat')
vfr_dat = sio.loadmat(vfr_f)['data']

In [112]:
from psifr import fr

In [107]:
list(list(vfr_dat[0, 0])[-1][0, 0])[0]

array([[array(['LTP001'], dtype='<U6'), array(['LTP001'], dtype='<U6'),
        array(['LTP001'], dtype='<U6'), ...,
        array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype=float64)],
       [array(['LTP001'], dtype='<U6'), array(['LTP001'], dtype='<U6'),
        array(['LTP001'], dtype='<U6'), ...,
        array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype=float64)],
       [array(['LTP001'], dtype='<U6'), array(['LTP001'], dtype='<U6'),
        array(['LTP001'], dtype='<U6'), ...,
        array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype=float64)],
       ...,
       [array(['LTP047'], dtype='<U6'), array(['LTP047'], dtype='<U6'),
        array(['LTP047'], dtype='<U6'), ...,
        array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype

In [53]:
vfr_dat[0, 0]

(array([[ 1],
       [ 1],
       [ 1],
       ...,
       [47],
       [47],
       [47]], dtype=uint8), array([[array(['LTP001'], dtype='<U6')],
       [array(['LTP001'], dtype='<U6')],
       [array(['LTP001'], dtype='<U6')],
       ...,
       [array(['LTP047'], dtype='<U6')],
       [array(['LTP047'], dtype='<U6')],
       [array(['LTP047'], dtype='<U6')]], dtype=object), array([[1],
       [1],
       [1],
       ...,
       [4],
       [4],
       [4]], dtype=uint8), array([[array(['ROOT'], dtype='<U4'), array(['LIP'], dtype='<U3'),
        array(['SPOOL'], dtype='<U5'), ...,
        array(['JEWEL'], dtype='<U5'), array(['SWAN'], dtype='<U4'),
        array(['COMPASS'], dtype='<U7')],
       [array(['FOOTBALL'], dtype='<U8'), array(['CREW'], dtype='<U4'),
        array(['CRACKER'], dtype='<U7'), ...,
        array(['VEST'], dtype='<U4'), array(['CUCUMBER'], dtype='<U8'),
        array(['ROBBER'], dtype='<U6')],
       [array(['SILVERWARE'], dtype='<U10'),
        array(['HEAD'],

In [14]:
vfr_dat

array([[(array([[ 1],
       [ 1],
       [ 1],
       ...,
       [47],
       [47],
       [47]], dtype=uint8), array([[array(['LTP001'], dtype='<U6')],
       [array(['LTP001'], dtype='<U6')],
       [array(['LTP001'], dtype='<U6')],
       ...,
       [array(['LTP047'], dtype='<U6')],
       [array(['LTP047'], dtype='<U6')],
       [array(['LTP047'], dtype='<U6')]], dtype=object), array([[1],
       [1],
       [1],
       ...,
       [4],
       [4],
       [4]], dtype=uint8), array([[array(['ROOT'], dtype='<U4'), array(['LIP'], dtype='<U3'),
        array(['SPOOL'], dtype='<U5'), ...,
        array(['JEWEL'], dtype='<U5'), array(['SWAN'], dtype='<U4'),
        array(['COMPASS'], dtype='<U7')],
       [array(['FOOTBALL'], dtype='<U8'), array(['CREW'], dtype='<U4'),
        array(['CRACKER'], dtype='<U7'), ...,
        array(['VEST'], dtype='<U4'), array(['CUCUMBER'], dtype='<U8'),
        array(['ROBBER'], dtype='<U6')],
       [array(['SILVERWARE'], dtype='<U10'),
        array([